In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import os

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "D:/IIT Docs/2nd Year/Semester 1/CM 2603 - Data Science Project/Data/Dataset/Potato/color",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [4]:
# Preprocess dataset
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
dataset = dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
# Split dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [6]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [7]:
# Create base model (MobileNetV2)
base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                         include_top=False,
                         weights='imagenet')

9406464/9406464 [==============================] - 68s 7us/step


In [8]:
# Freeze base model
base_model.trainable = False

In [9]:
# Add classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')

In [10]:
# Build the model
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [12]:
# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=10)

Epoch 1/10
54/54 [==============================] - 25s 367ms/step - loss: 0.4185 - accuracy: 0.8594 - val_loss: 0.1931 - val_accuracy: 0.9612
Epoch 2/10
54/54 [==============================] - 19s 345ms/step - loss: 0.1487 - accuracy: 0.9554 - val_loss: 0.1350 - val_accuracy: 0.9698
Epoch 3/10
54/54 [==============================] - 19s 347ms/step - loss: 0.1068 - accuracy: 0.9734 - val_loss: 0.1091 - val_accuracy: 0.9784
Epoch 4/10
54/54 [==============================] - 19s 346ms/step - loss: 0.0833 - accuracy: 0.9763 - val_loss: 0.0947 - val_accuracy: 0.9741
Epoch 5/10
54/54 [==============================] - 19s 346ms/step - loss: 0.0699 - accuracy: 0.9826 - val_loss: 0.0831 - val_accuracy: 0.9784
Epoch 6/10
54/54 [==============================] - 19s 345ms/step - loss: 0.0556 - accuracy: 0.9861 - val_loss: 0.0753 - val_accuracy: 0.9784
Epoch 7/10
54/54 [==============================] - 19s 346ms/step - loss: 0.0489 - accuracy: 0.9902 - val_loss: 0.0719 - val_accuracy: 0.9828

In [14]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')
print("Accuracy: ", accuracy * 100, "%")

8/8 [==============================] - 3s 287ms/step - loss: 0.0521 - accuracy: 0.9883
Test accuracy: 0.98828125
Accuracy:  98.828125 %


In [16]:
# Save the model
model_version = max([int(i) for i in os.listdir("mobilenet_models") + [0]]) + 1
model.save(f"mobilenet_models/{model_version}")

INFO:tensorflow:Assets written to: mobilenet_models/1\assets


INFO:tensorflow:Assets written to: mobilenet_models/1\assets


In [17]:
model.save("potato_mobilenetv2_model.h5")

D:\IIT Docs\2nd Year\Semester 1\CM 2603 - Data Science Project\Project\PotatoEarlyBlightDetection\PotatoEarlyBlightDetectionProject\.venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# from pathlib import Path
# 
# model_structure = model.to_json()
# 
# f = Path("potato_model_structure_mobilenetv2.json")
# f.write(model_structure)